In [ ]:
import json
import os
import pickle
import re
import time
from pathlib import Path
import requests

import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from nltk import download
from nltk.tokenize import word_tokenize
import torch
from tqdm.auto import tqdm
from transformers import MBart50Tokenizer, MBartForConditionalGeneration
from transformers import pipeline

download('punkt')

### Konstruktikon:

In [ ]:
def parse_sentences(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    construction_id = root.attrib.get('id')
    try:
        category, name = root.attrib.get('name').split(':')
    except ValueError:
        category = root.attrib.get('name')
        name = None

    sentences_data = []
    
    for sentence in root.findall('.//sentence'):
        if sentence.attrib.get('uid') is None:
            continue
        uid = sentence.attrib.get('uid')
        text = sentence.find('.//text').text.strip()
        contextleft = sentence.find('.//contextleft').text.strip()
        contextright = sentence.find('.//contextright').text.strip()
        
        sentences_data.append({
            'uid': uid,
            'text': text,
            'contextleft': contextleft,
            'contextright': contextright,
            'construction_id': int(construction_id),
            'category': category,
            'name': name,
        })

    return sentences_data

kee_list = []
xml_directory = '../../data/constructicon/construction'

for filename in tqdm(list(os.listdir(xml_directory))):
    if filename.endswith('.xml'):
        constr_id = Path(filename).stem
        if b"fa-triangle-exclamation" in requests.get(f"https://gsw.phil.hhu.de/constructicon/construction?id={constr_id}").content:
            print(constr_id, "does not exist online!")
            continue
        xml_file = os.path.join(xml_directory, filename)
        data = parse_sentences(xml_file)
        if data:
            kee_list += data
        time.sleep(.5)

sentences = pd.DataFrame.from_dict(kee_list)
sentences.set_index('uid', inplace=True)
sentences

In [ ]:
constructions = sentences[['construction_id', 'category', 'name']].drop_duplicates()
constructions.set_index('construction_id', inplace=True)
constructions

In [ ]:
constructions['category'].value_counts()

In [ ]:
def parse_kee(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    construction_id = root.attrib.get('id')

    ke_data = []
    
    for ke in root.findall('.//ke'):
        ke_data.append({
            'ke_id': int(ke.attrib.get("id")),
            'ke_name': ke.attrib.get("name"),
            'ke_coretype': ke.attrib.get("coretype"),
            'ke_attributes': ke.attrib.get("attributes")
        })

    kee_data = []
    
    for kee in root.findall('.//kee'):
        for ke in ke_data:
            kee_data.append({
                'kee_id': int(kee.attrib.get("id")),
                'kee_name': kee.attrib.get("name"),
                'construction_id': int(construction_id)
            } | ke)

    return kee_data

kee_list = []
xml_directory = '../../data/constructicon/construction'

for filename in tqdm(list(os.listdir(xml_directory))):
    if filename.endswith('.xml'):
        constr_id = Path(filename).stem
        if b"fa-triangle-exclamation" in requests.get(f"https://gsw.phil.hhu.de/constructicon/construction?id={constr_id}").content:
            print(constr_id, "does not exist online!")
            continue
        xml_file = os.path.join(xml_directory, filename)
        data = parse_kee(xml_file)
        if data:
            kee_list += data
        time.sleep(.5)

kees = pd.DataFrame.from_dict(kee_list)
kees.set_index('kee_id', inplace=True)
kees

In [ ]:
kees['ke_name'].value_counts()

In [ ]:
kees['kee_name'].value_counts()

Die KEEs sind markiert wie folgt:

```
            <annotationset>
            ...
                <layer name="KEE-Negation:NEG_X_geschweige_denn_Y" descriptor="KEE-Negation:NEG_X_geschweige_denn_Y">
                    <label start="72" end="87" name="geschweige_denn" refid="GC_KEE:1" itype="null" groupid="0"></label>
                </layer>
            ...
            </annotationset>
```

In [ ]:
def parse_masked_sentences(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    sentences_data = []
    
    constr_id = int(root.attrib.get('id'))
    
    for sentence in root.findall('.//sentence'):
        if sentence.attrib.get('uid') is None:
            continue
        
        uid = sentence.attrib.get('uid')
        text = sentence.find('.//text').text.strip()
        
        text_pos = []
        kees = []
        kees_idx = []
        kes = []
        kes_idx = []
        
        # Loop through annotations:
        for layer in sentence.findall('.//layer'):
            layer_name = layer.attrib.get('name')
            
            # Get the KEE annotations:
            if "KE-" in layer_name or "KEE-" in layer_name:
                # Loop over all KEEs or KEs:
                for label in layer.findall('.//label'):
                    start = int(label.attrib.get('start'))
                    end = int(label.attrib.get('end'))
                    if "KEE-" in layer_name:
                        kees.append(text[start:end])  # Read the KEE
                        kees_idx.append((start, end))  # Log the position of the KEE
                    else:
                        kes.append(text[start:end])  # Read the KE
                        kes_idx.append((start, end))  # Log the position of the KE
            elif "UPOS" == layer_name:
                for label in layer.findall('.//label'):
                    text_pos.append(label.attrib.get('name'))
        
        sentences_data.append({
            'uid': uid,
            'constr_id': constr_id,
            'text': text,
            'text_pos': text_pos,
            'kees': kees,
            'kees_idx': kees_idx,
            'kes': kes,
            'kes_idx': kes_idx,
        })
    return sentences_data
    
sentence_list = []
xml_directory = ('../../data/constructicon/construction')

for filename in tqdm(list(os.listdir(xml_directory))):
    if filename.endswith('.xml'):
        constr_id = Path(filename).stem
        if b"fa-triangle-exclamation" in requests.get(f"https://gsw.phil.hhu.de/constructicon/construction?id={constr_id}").content:
            print(constr_id, "does not exist online!")
            continue
        xml_file = os.path.join(xml_directory, filename)
        data = parse_masked_sentences(xml_file)
        if data:
            sentence_list += data
        time.sleep(.5)

sentences = pd.DataFrame.from_dict(sentence_list)
# sentences.set_index('uid', inplace=True)
sentences

In [ ]:
sentences = sentences.explode(["kees", "kees_idx"], ignore_index=True)
sentences

In [ ]:
sentences = sentences.explode(["kes", "kes_idx"], ignore_index=True)
sentences

In [ ]:
json_comapp = []
csv_comapp = []
errors = 0
problematic_constructions = set()
unproblematic_constructions = set()


for _, row in tqdm(sentences.iterrows(), total=len(sentences)):
    tokenized = str(row["text"]).split()  # tokenizer.tokenize(...)
    
    try:
        ke_start, ke_end = row["kes_idx"]
        tokenized_kees = str(row["kees"]).split()  # tokenizer.tokenize(...)  # Split the KEEs if there are multi-word KEEs
    except TypeError:
        continue  # If there is nothing to mask or if there is no KEE, we can't use this example.
        
    masked_text = row["text"][:ke_start] + "<mask>" + row["text"][ke_end:]
    tokenized_masked = str(masked_text).split()
    # Rejoin "<", "mask" and ">".
    tokenized_masked = [
        '<mask>' if tokenized_masked[i] == '<' else tokenized_masked[i]  # add "<mask>" back in instead of "<"
        for i in range(len(tokenized_masked)) 
        if tokenized_masked[i] != 'mask' and tokenized_masked[i] != '>'  # remove "mask" and ">"
    ]
    
    try:
        kee_idx = [tokenized.index(tokenized_kee) for tokenized_kee in tokenized_kees]
        kee_query_idx = []
        for i in kee_idx:
            if tokenized_masked.index("<mask>") < i:  # indicates that the query_idx might have been shifted because the multi-token mask comes first
                offset = len(tokenized) - len(tokenized_masked)
                assert i - offset >= 0
                kee_query_idx.append(i - offset)
            else:
                kee_query_idx.append(i)
    except (ValueError, AssertionError) as e:
        print(row["constr_id"], type(e), e, "... Continuing ...")
        errors += 1
        problematic_constructions.add(row["constr_id"])
        continue
    
    unproblematic_constructions.add(row["constr_id"])
    
    out_json = [{
        "label": kee + str(row["constr_id"]),
        "target1": row["text"], 
        "target1_idx": idx, 
        "query": masked_text,
        "query_idx": q
    } for kee, idx, q in zip(tokenized_kees, kee_idx, kee_query_idx)]  # Split the KEEs if there are multi-word KEEs
    out_csv = [{
        "text": row["text"],
        "pos_tags": row["text_pos"],
        "mask": row["text"][ke_start:ke_end],
        "ambiguous_word": kee,
        "label": kee + str(row["constr_id"])  # This will be the new token that we will add to the LLM (named "<kee><i>" where <kee> and <i> are replaced by the KEE's name and i will be replaced by the construction it appears in).
    } for kee in tokenized_kees]
    
    json_comapp += out_json
    csv_comapp += out_csv
    
with open("../../data/pseudowords/CoMaPP_all.json", "w") as file:
    json.dump(json_comapp, file, ensure_ascii=False)

f"{errors} elements from {len(problematic_constructions)} different constructions could not be saved as intended."

In [ ]:
import csv

with open("../../data/pseudowords/CoMapp_Dataset.csv", "w+", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["label", "text", "pos_tags", "mask", "ambiguous_word"])
    writer.writeheader()
    writer.writerows(csv_comapp)

In [ ]:
# Load the pre-trained MBart-50 model and tokenizer
model_name = "facebook/mbart-large-50"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="de_DE", tgt_lang="de_DE")

In [ ]:
# Define a masked input sequence and convert it into tokens
masked_sequence = "Ich bin <mask> gegangen."
input_ids = tokenizer.encode(masked_sequence, return_tensors="pt")

In [ ]:
# Generate predictions
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=100, num_return_sequences=15, num_beams=20)

In [ ]:
predicted_texts = []
for output in outputs:
    predicted_texts.append(tokenizer.decode(output, skip_special_tokens=True))
predicted_texts

In [ ]:
# Create a dataframe which matches the index of the constructions dataframe and the constr_id of the kees dataframe
kee_categories = sentences.join(constructions, on="constr_id")[["constr_id", "kees", "category", "name"]].drop_duplicates().reset_index(drop=True)
kee_categories

In [ ]:
# Count the distinct names:
kee_categories["name"].value_counts()

Vergleiche wie ähnlich sich cased und uncased sind:

In [ ]:
input_text_1 = "Wie Schulbücher den Mauerfall darstellen"
input_text_2 = "wie Schulbücher den Mauerfall darstellen"

input_tokens_1 = tokenizer(input_text_1, return_tensors="pt", padding=True, truncation=True)
input_tokens_2 = tokenizer(input_text_2, return_tensors="pt", padding=True, truncation=True)

torch.stack((input_tokens_1.input_ids, input_tokens_2.input_ids))

In [ ]:
with torch.no_grad():
    model_output_1 = model(**input_tokens_1)
    model_output_2 = model(**input_tokens_2)
    
hidden_states_1 = model_output_1.encoder_last_hidden_state
hidden_states_2 = model_output_2.encoder_last_hidden_state

sequence_emb_1 = hidden_states_1[:, 5, :]
sequence_emb_2 = hidden_states_2[:, 5, :]

sequence_emb_1, sequence_emb_2

In [ ]:
# TODO Markiere die zu desambiguierenden Wörter
# Position ergibt keinen Sinn, weil die maskierten Bereiche beliebig lang sein können (und damit der Index variabel ist in Abhängigkeit der länge der generierten Phrase.
# Idee wäre festgelegtes Symbol (z. B. §) oder gar kein Symbol und einfach heuristisch von .index(wort) ausgehen.

In [ ]:
from transformers import Text2TextGenerationPipeline, MBartForConditionalGeneration
from transformers import MBart50Tokenizer

model_name = "facebook/mbart-large-50"  # "facebook/mbart-large-cc25" 

model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="de_DE", tgt_lang="de_DE")

nlp = Text2TextGenerationPipeline(model=model, tokenizer=tokenizer, device=0)
masked_sequence = "Ich bin <mask> gegangen."

predicted_texts = nlp(masked_sequence, max_length=100, num_return_sequences=5, num_beams=20)
predicted_texts = [result['generated_text'] for result in predicted_texts]

print(predicted_texts)

In [ ]:
x = tokenizer("Moritz sitzt neben mir.", text_target="Moritz sitzt neben mir.")

tokenizer.decode(x.input_ids)

In [ ]:
from transformers import BartTokenizer

tokenizer2 = BartTokenizer.from_pretrained("facebook/bart-large")

x = tokenizer2("Moritz sitzt neben mir.", text_target="Moritz sitzt neben mir.")

tokenizer2.decode(x.input_ids), tokenizer2.decode(x.labels)

Tests mit LLMs (Llama-2-13B-German-Assistant-v4-GPTQ): Hierzu müssen die Definitionen der Konstruktionen geladen werden.

In [ ]:
def parse_definitions(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    #definition = {
    #    "constr_id": root.attrib.get('id'),
    #    "definition": root.find('.//definition').text.strip()
    #}
    definition = root.find('.//definition').text.strip()
    return definition

xml_directory = '../../data/constructicon/construction'

definitions = []

for filename in tqdm(list(os.listdir(xml_directory))):
    if filename.endswith('.xml'):
        constr_id = Path(filename).stem
        if b"fa-triangle-exclamation" in requests.get(f"https://gsw.phil.hhu.de/constructicon/construction?id={constr_id}").content:
            print(constr_id, "does not exist online!")
            continue
        xml_file = os.path.join(xml_directory, filename)
        data = parse_definitions(xml_file)
        if data:
            definitions.append(data)
        time.sleep(.25)

In [ ]:
definitions

In [ ]:
definitions_clean = [re.sub("\[/?\w+::[\w-]+\]", '"', d) for d in definitions]
definitions_clean = [re.sub("\*\*", '"', d) for d in definitions_clean]
definitions_clean = [re.sub("\_\_+", '', d) for d in definitions_clean]

definitions_clean

In [ ]:
import pickle

with open("../../out/definitions.pickle", "wb") as file:
    pickle.dump(definitions_clean, file)